In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

data_dir = "/content/drive/MyDrive/daa"  # 데이터 경로
human_poems_dir = os.path.join(data_dir, "human")
ai_poems_dir = os.path.join(data_dir, "ai")

human_poems = [open(os.path.join(human_poems_dir, file)).read() for file in os.listdir(human_poems_dir)]
ai_poems = [open(os.path.join(ai_poems_dir, file)).read() for file in os.listdir(ai_poems_dir)]

poems = human_poems + ai_poems
labels = [1] * len(human_poems) + [0] * len(ai_poems)

X_train, X_test, y_train, y_test = train_test_split(poems, labels, test_size=0.2, random_state=42)

max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

# Save tokenizer configuration
tokenizer_config_path = '/content/drive/MyDrive/daa/tokenizer_config.npy'
np.save(tokenizer_config_path, {'word_index': tokenizer.word_index, 'document_count': tokenizer.document_count})

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_pad, np.array(y_train), epochs=500, batch_size=16, validation_split=0.2)

loss, accuracy = model.evaluate(X_test_pad, np.array(y_test))
print(f'Test accuracy: {accuracy}')

# Save the model
model.save('/content/drive/MyDrive/daa/my_poem_model')


Epoch 1/500
18/18 [==============================] - 7s 167ms/step - loss: 0.6957 - accuracy: 0.5267 - val_loss: 0.6914 - val_accuracy: 0.5070
Epoch 2/500
18/18 [==============================] - 3s 165ms/step - loss: 0.6784 - accuracy: 0.5267 - val_loss: 0.6685 - val_accuracy: 0.5352
Epoch 3/500
18/18 [==============================] - 3s 194ms/step - loss: 0.4360 - accuracy: 0.9573 - val_loss: 0.2582 - val_accuracy: 0.9155
Epoch 4/500
18/18 [==============================] - 3s 153ms/step - loss: 0.0568 - accuracy: 0.9822 - val_loss: 0.3843 - val_accuracy: 0.9155
Epoch 5/500
18/18 [==============================] - 2s 125ms/step - loss: 0.0230 - accuracy: 0.9858 - val_loss: 0.4562 - val_accuracy: 0.9155
Epoch 6/500
18/18 [==============================] - 3s 171ms/step - loss: 0.0173 - accuracy: 0.9964 - val_loss: 0.5108 - val_accuracy: 0.9014
Epoch 7/500
18/18 [==============================] - 2s 127ms/step - loss: 0.0155 - accuracy: 0.9929 - val_loss: 0.5117 - val_accuracy: 0.9014

In [ ]:

user_poem = input("예측하고 싶은 시를 입력하세요: ")

user_poem_seq = tokenizer.texts_to_sequences([user_poem])
user_poem_pad = pad_sequences(user_poem_seq, maxlen=max_len)
prediction = model.predict(user_poem_pad)[0, 0]

if prediction > 0.5:
    print(f"예측 결과: 인간의 시로 판단됩니다. (확률: {prediction:.4f})")
else:
    print(f"예측 결과: AI가 생성한 시로 판단됩니다. (확률: {1 - prediction:.4f})")


예측하고 싶은 시를 입력하세요: 하이
1/1 [==============================] - 0s 464ms/step
예측 결과: 인간의 시로 판단됩니다. (확률: 0.6730)
